<a href="https://colab.research.google.com/github/ramonis567/SupervisedML-Training/blob/master/Machine_Learning_WiDS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Supervised Machine Learning**
---


#### **Conteúdo:**
 - **Aprendizados:** Machine Learning Supervisionado
 - **Algoritmos de Classificação:** Regressão Logistíca, K-NN, Naive Bayes, SVM, Árvores de Decisão
 - **Aplicação dos Algoritmos**
 - **Validação dos Algoritmos**

**Importação de Dependências**

A seguir são importadas todas as dependências que serão utilizadas.

* Bibliotecas Pandas, NumPy e Scikit-Learn
* Arquivo .csv com os dados que serão utilizados
* O Dataset utilizado é o Titanic Dataset

Esse conjunto de dados contém 11 váriaveis, que podem ser utilizados para predizer se a pessoa vai sobreviver ao acidente ou não.

In [31]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
titanic = pd.read_csv("https://raw.githubusercontent.com/ramonis567/SupervisedML-Training/master/train.csv")
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


**Agora, as váriaveis numéricas estão sendo divididas das váriaveis categóricas.**


In [32]:
titanic_num = titanic.select_dtypes(np.number)
titanic_num.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,0,3,22.0,1,0,7.2500
1,2,1,1,38.0,1,0,71.2833
2,3,1,3,26.0,0,0,7.9250
3,4,1,1,35.0,1,0,53.1000
4,5,0,3,35.0,0,0,8.0500


In [33]:
titanic_cat = titanic.select_dtypes(object)
titanic_cat.head()

,Name,Sex,Ticket,Cabin,Embarked
0,"Braund, Mr. Owen Harris",male,A/5 21171,NaN,S
1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,PC 17599,C85,C
2,"Heikkinen, Miss. Laina",female,STON/O2. 3101282,NaN,S
3,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,113803,C123,S
4,"Allen, Mr. William Henry",male,373450,NaN,S


**Agora, as váriaveis não informativas, serão removidas.**

In [34]:
titanic_cat.drop(['Name','Ticket'], axis=1, inplace=True)
titanic_cat.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Sex,Cabin,Embarked
0,male,NaN,S
1,female,C85,C
2,female,NaN,S
3,female,C123,S
4,male,NaN,S


**Avaliação de valores nulos.**

In [35]:
titanic_cat.isnull().sum()

Sex           0
Cabin       687
Embarked      2
dtype: int64

**A váriavel "*Cabin*" tem 687 valores nulos de um total de 891 (77%).**

Assim, temos 3 possibilidades de ação para a váriavel:
- Deletar todas as linhas em que a váriavel é nula
 - Resultado: Perca de 77% dos dados do dataset
- Deletar a váriavel
 - Resultado: Perca de uma váriavel importante na predição
- **Substituir a váriavel pela contagem máxima da categoria**
 - **Melhor escolha**

In [36]:
titanic_cat.Cabin.fillna(titanic_cat.Cabin.value_counts().idxmax(), inplace=True)
titanic_cat.Embarked.fillna(titanic_cat.Embarked.value_counts().idxmax(), inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


**Uso da LabelEncoder para associar as váriaveis categóricas a valores numéricos.**

In [37]:
le = LabelEncoder()
titanic_cat = titanic_cat.apply(le.fit_transform)
titanic_cat.head()

,Sex,Cabin,Embarked
0,1,145,2
1,0,81,0
2,0,145,2
3,0,55,2
4,1,145,2


**Avaliar valores nulos em váriaveis numéricas.**

In [38]:
titanic_num.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Age            177
SibSp            0
Parch            0
Fare             0
dtype: int64

Substituição dos valores faltantes na váriavel "age" pela  média de idade.

In [39]:
titanic_num.Age.fillna(titanic_num.Age.mean(), inplace=True)
titanic_num.isna().sum()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


PassengerId    0
Survived       0
Pclass         0
Age            0
SibSp          0
Parch          0
Fare           0
dtype: int64

**Remoção da váriavel PassengerId, pois não é informativa.**

In [40]:
titanic_num.drop(['PassengerId'], axis=1, inplace=True)
titanic_num.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Survived,Pclass,Age,SibSp,Parch,Fare
0,0,3,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,1,3,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,0,3,35.0,0,0,8.0500


**Concatenando os dados categóricos e numéricos novamente.**

In [41]:
titanic_final = pd.concat([titanic_cat,titanic_num],axis=1)
titanic_final.head()

,Sex,Cabin,Embarked,Survived,Pclass,Age,SibSp,Parch,Fare
0,1,145,2,0,3,22.0,1,0,7.2500
1,0,81,0,1,1,38.0,1,0,71.2833
2,0,145,2,1,3,26.0,0,0,7.9250
3,0,55,2,1,1,35.0,1,0,53.1000
4,1,145,2,0,3,35.0,0,0,8.0500


**Divisão dos dados de treino e teste.**

In [42]:
X = titanic_final.drop(['Survived'], axis=1)
Y = titanic_final['Survived']

In [43]:
X_train = np.array(X[0:int(0.80*len(X))])
Y_train = np.array(Y[0:int(0.80*len(Y))])
X_test = np.array(X[int(0.80*len(X)):])
Y_test = np.array(Y[int(0.80*len(Y)):])
len(X_train), len(Y_train), len(X_test), len(Y_test)

(712, 712, 179, 179)

* **712 indivíduos para Treino.**
* **179 indivíduos para Teste.**

**Inicialização de algoritmos:**

In [44]:
LR = LogisticRegression()
KNN = KNeighborsClassifier()
NB = GaussianNB()
LSVM = LinearSVC()
DT = DecisionTreeClassifier()

**Hora de treinar os algoritmos com os dados!!**

In [45]:
LR_fit = LR.fit(X_train, Y_train)
KNN_fit = KNN.fit(X_train, Y_train)
NB_fit = NB.fit(X_train, Y_train)
LSVM_fit = LSVM.fit(X_train, Y_train)
DT_fit = DT.fit(X_train, Y_train)


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


**Predição da sobrevivência no conjunto de teste.**

In [46]:
LR_pred = LR_fit.predict(X_test)
KNN_pred = KNN_fit.predict(X_test)
NB_pred = NB_fit.predict(X_test)
LSVM_pred = LSVM_fit.predict(X_test)
DT_pred = DT_fit.predict(X_test)

**Acurácia dos modelos:**

In [47]:
from sklearn.metrics import accuracy_score
print("Logistic Regression is %f percent accurate" % (accuracy_score(LR_pred, Y_test)*100))
print("KNN is %f percent accurate" % (accuracy_score(KNN_pred, Y_test)*100))
print("Naive Bayes is %f percent accurate" % (accuracy_score(NB_pred, Y_test)*100))
print("Linear SVMs is %f percent accurate" % (accuracy_score(LSVM_pred, Y_test)*100))
print("Decision Trees is %f percent accurate" % (accuracy_score(DT_pred, Y_test)*100))


Logistic Regression is 82.122905 percent accurate
KNN is 78.212291 percent accurate
Naive Bayes is 81.005587 percent accurate
Linear SVMs is 67.039106 percent accurate
Decision Trees is 79.888268 percent accurate
